In [5]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import sys
import time

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 
# load dataset
dataset = read_csv('alya_formatted_2.csv', header=0, index_col=0)
values = dataset.values

n_days_predict=5
n_features=5

# ensure all data is float
values = values.astype('float64')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# frame as supervised learning
reframed = series_to_supervised(scaled, n_days_predict, 1)

# drop columns we don't want to predict
# reframed.drop(reframed.columns[[6,7,8,9]], axis=1, inplace=True)
# print(reframed.head())

# split into train and test sets
values = reframed.values
n_train_hours = 90 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_days_predict * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

#print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

#print shape of tensors
#print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
print ('Training set:')
print (len(train_X))
print ('Test set: ')
print (len(test_X))

time.sleep(5.5)

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=200, batch_size=50, validation_data=(test_X, test_y), verbose=2, shuffle=False)



dataset = read_csv('alya_formatted_2.csv', header=0, index_col=0)
values = dataset.values

n_days_predict=5
n_features=5

# Getting relevant feature
# real_price = dataset.iloc[:,1:2]
# real_value = real_price.value()


#y
#yhat inverse transform

Training set:
2160
Test set: 
821
Train on 2160 samples, validate on 821 samples
Epoch 1/200
 - 1s - loss: 0.1133 - val_loss: 0.3606
Epoch 2/200
 - 0s - loss: 0.0834 - val_loss: 0.2128
Epoch 3/200
 - 0s - loss: 0.0482 - val_loss: 0.1453
Epoch 4/200
 - 0s - loss: 0.0389 - val_loss: 0.1049
Epoch 5/200
 - 0s - loss: 0.0290 - val_loss: 0.0359
Epoch 6/200
 - 0s - loss: 0.0180 - val_loss: 0.0565
Epoch 7/200
 - 0s - loss: 0.0247 - val_loss: 0.0352
Epoch 8/200
 - 0s - loss: 0.0135 - val_loss: 0.0564
Epoch 9/200
 - 0s - loss: 0.0278 - val_loss: 0.0573
Epoch 10/200
 - 0s - loss: 0.0156 - val_loss: 0.0155
Epoch 11/200
 - 0s - loss: 0.0118 - val_loss: 0.0215
Epoch 12/200
 - 0s - loss: 0.0137 - val_loss: 0.0295
Epoch 13/200
 - 0s - loss: 0.0188 - val_loss: 0.0632
Epoch 14/200
 - 0s - loss: 0.0230 - val_loss: 0.0161
Epoch 15/200
 - 0s - loss: 0.0099 - val_loss: 0.0313
Epoch 16/200
 - 0s - loss: 0.0153 - val_loss: 0.0510
Epoch 17/200
 - 1s - loss: 0.0230 - val_loss: 0.0401
Epoch 18/200
 - 0s - loss: 

AttributeError: 'DataFrame' object has no attribute 'value'